## Esercizio: 
Si estenda il linguaggio MiniCaml visto a lezione con il costrutto CodaLimitata per la definizione di code
con lunghezza massima prefissata. In aggiunta, il linguaggio è esteso con le operazioni primitive `insert` e
`remove`, che rispettano la politica FIFO, e `peek`, che restituisce l’elemento in cima alla coda. Si mostri come
deve essere modificato l’interprete OCaml del linguaggio

### Soluzione:

Per qualsiasi estensione di minicaml, **la funzione `eval` è la prima a cui pensare, sebbene sia l'ultima da scrivere**. In questo caso, vogliamo che `eval` assegni un valore alle nuove espressioni su queue, quindi ci serviranno dei nuovi nodi dell'AST (dentro `exp`) e dei nuovi valori di MiniCaml dentro (`evT`). **Precisazione**, non è sempre questo il caso. Se dovessimo ad esempio estendere MiniCaml con un costrutto

    if(guard) exp elif (guard) exp  ... elif(guard) exp else exp
vogliamo che `eval` valuti le guardie ed esegua l'espressione corrispondente. In questo caso non abbiamo bisogno di estendere `evT`, perchè non c'è nessun nuovo valore che MiniCaml può trattare

Ritornando al nostro esercizio, dobbiamo aggiungere all'AST tutte le espressioni necessarie per la creazione, inserzione, rimozione e peek su code. Possiamo immaginare che la lunghezza massima di una queue venga determinata dall'utente al momento della creazione, e quindi definiamo il costrutto `EQueue of exp`, dove exp sarà la nostra lunghezza massima. Questo costrutto insomma permette di creare code vuote con una lunghezza massima prefissata. Un'alternativa postrebbe essere `EQueue of exp * exp list`, dove la creazione di una coda contiene sia la lunghezza massima che il suo contenuto iniziale. Nel dubbio, seguiamo l'alternativa più semplice.

In [ ]:
type exp = 
| ...
| EQueue of exp
| EPeek of exp
| ERemove of exp
| EInsert of exp * exp

Come detto, dobbiamo aggiungere un nuovo valore dentro `evT`, che rappresenti una queue di valori con lunghezza massima prefissata. Aggiungiamo anche il corrispettivo tipo di MiniCaml, e aggiorniamo `gettype` e `typecheck`.

Ipotizziamo che le primitive `insert`, `peek` e `remove` siano già implementate. Durante il compito, chiedete sempre se dovete implementare le primitive o meno.

In [ ]:
type evT = 
| ...
| Queue of int * evT list

type tname = ... | TQueue

let gettype v = 
    match v with
    | ... 
    | Queue -> TQueue

let typecheck (v, t) = 
    match t with
    | ... 
    | TQueue -> ( match v with
                  Queue _ -> true
                  _ -> false)

Per la funzione di valutazione, dobbiamo fare caso a:
- Nell'espressione `EQueue(eSize)`, dobbiamo controllare che `eSize` valuti ad un intero, e associarle una queue vuota.
- Nell'espressione `EInsert(eQueue, eEl)` dobbiamo controllare che `eQueue` valuti ad una queue, e che tale queue non sia già piena.
- Nelle espressioni `EPeek` e `ERemove` dobbiamo solo controllare che `eQueue` valuti ad una queue. In alternativa, potremmo anche direttamente chiamare la primitiva corrispondente, scrivendo a caratteri cubitali: 

    > "Assumo che la primitiva esegua anche le necessarie operazioni di typechecking, ad esempio chiamando `typecheck(TQueue, v)`"

   In un modo o nell'altro, rendete sempre chiaro che sapete come funziona l'interprete, non scrivete semplicemente "questa parte è già implementata".

In [ ]:
let rec eval e s = 
    match e with 
    | ... -> 
    |EQueue(eSize) -> 
        let size = eval eSize s in
        if typecheck(TInt, size)
            then Queue(size, [])
            else raise Error("Queue size must be integer")
    |EPeek(eQueue) -> 
        let queue = eval eQueue s in
        if typecheck(TQueue, queue)
            then peek(queue)
            else raise Error("Wrong type")
    |ERemove(eQueue) -> 
        let queue = eval eQueue s in
        if typecheck(TQueue, queue)
            then remove(queue)
            else raise Error("Wrong type")
    |EInsert(eQueue, eEl) -> 
        let queue = eval eQueue s in
        let el = eval eEl s in
        match queue with
        |Queue(n, l) ->
            if List.length l < n 
                then insert(queue, el)
                else raise Error("Full Queue")
        | _ -> raise Error("Wrong type")